# Lista 7

In [17]:
import pywt
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from scipy.signal import chirp, square, sawtooth

### Zadanie 1
Przygotuj kod w Pythonie, który wyświetli następujące typy falek: Haar, Daubechies, Symlets, Coiflets, Biortogonalna, Gaussian, Meksykański kapelusz, Morleta. W celu rozwiązania zadania można wykorzystać pakiet pywt

In [ ]:
def draw_wavelet(wavelet, label, level=5):
    try:
        # Pobierz dane falki
        wavelet = pywt.Wavelet(wavelet)
        result = wavelet.wavefun(level=level)

    except ValueError:
        wavelet = pywt.ContinuousWavelet(wavelet)
        result = wavelet.wavefun(level=level)

    plt.figure(figsize=(10, 4))

    if len(result) == 3:
        phi, psi, x = result

        plt.subplot(1, 2, 1)
        plt.plot(x, phi)
        plt.title('Funkcja skalująca (phi)')
        plt.grid()

        plt.subplot(1, 2, 2)
        plt.plot(x, psi)
        plt.title('Funkcja falkowa (psi)')
        plt.grid()
        plt.suptitle(f'Falka: {label}')
        plt.tight_layout()

    if len(result) == 2:
        psi, x = result

        plt.plot(x, psi)
        plt.title('Funkcja falkowa (psi)')
        plt.grid()
        plt.suptitle(f'Falka: {label}')
        plt.tight_layout()





falki = {
    'Haar' : 'haar',
    'Daubechies 2' : 'db2',
    'Symlets 2' : 'sym2',
    'Coiflets 2' : 'coif2',
    'Biortogonalna' : 'bior1.3',
    'Gaussian 2' : 'gaus2',
    'Meksykański kapelusz' : 'mexh',
    'Morleta' : 'morl'
}


lvl_slider = widgets.IntSlider(min=2, max=15, step=1, value=5, description="stopień")

for label, wavelet in falki.items():
    display(widgets.interactive(draw_wavelet, label=widgets.fixed(label), wavelet=widgets.fixed(wavelet), level=lvl_slider))

### Zadanie 2
Przygotuj kod w Pythonie, który wyświetli falkę Daubechies w różnych wersjach (db1, db2, itd.) i dla różnych parametrów. 

In [ ]:
wersje = {
    'Daubechies' : {'Daubechies 1' : 'db1', 'Daubechies 2' : 'db2', 'Daubechies 3' : 'db3', 'Daubechies 4' : 'db4'},
    'Symlets' : { 'Symlets 2' : 'sym2', 'Symlets 3' : 'sym3',  'Symlets 4' : 'sym4'},
    'Coiflets' : {'Coiflets 1' : 'coif1', 'Coiflets 2' : 'coif2', 'Coiflets 3' : 'coif3', 'Coiflets 4' : 'coif4'}
}



for wersja in wersje.values():
    lvl_slider = widgets.IntSlider(min=2, max=15, step=1, value=5, description="stopień")
    for label, wavelet in wersja.items():
        display(widgets.interactive(draw_wavelet, label=widgets.fixed(label), wavelet=widgets.fixed(wavelet), level=lvl_slider))

### Zadanie 3
Przygotuj kod w Pythonie, który dokona dekompozycji sygnału świergotliwego (chirp signal) z wykorzystaniem trzech różnych falek. Uzyskane wyniki wyświetl w czytelnej postaci. 

In [19]:
def generate_signals(f=5, fs=1000):
    # Parametry sygnału
    t = np.linspace(0, 1, fs, endpoint=False)  # Oś czasu 1 sekunda

    # a) Sygnał sinusoidalny
    sin_wave = np.sin(2 * np.pi * f * t)

    # b) Sygnał prostokątny
    square_wave = square(2 * np.pi * f * t)  

    # c) Sygnał piłokształtny
    sawtooth_wave = sawtooth(2 * np.pi * f * t)  

    # d) Sygnał świergotliwy (chirp)
    chirp_wave = chirp(t, f0=f, f1=5*f, t1=1, method='linear')

    return t, [sin_wave, square_wave, sawtooth_wave, chirp_wave]

In [23]:
def decomposition_with_wavelet(f, wavelet, label):
    t, signals = generate_signals(f=f)
    names = ['sinus', 'prostokątny', 'piłokształtny', 'świergotliwy']

    for x, signal in enumerate(signals):
        # Wybór poziomu
        max_level = pywt.dwt_max_level(len(signal), pywt.Wavelet(wavelet).dec_len)
        level = min(4, max_level)

        # Dekompozycja
        coeffs = pywt.wavedec(signal, wavelet, level=level)

        # coeffs to lista: [A_n, D_n, D_{n-1}, ..., D1]
        # A - aproksymacja, D - szczegóły

        # Rysowanie
        plt.figure(figsize=(10, 8))
        plt.subplot(level + 2, 1, 1)
        plt.plot(signal)
        plt.title('Oryginalny sygnał')

        for i, c in enumerate(coeffs):
            plt.subplot(level + 2, 1, i + 2)
            plt.plot(c)
            if i == 0:
                plt.title(f'Aproksymacja A{level}')
            else:
                plt.title(f'Szczegóły D{level - i + 1}')

        plt.suptitle(f'Falka: {label}. Sygnał {names[x]}')
        plt.tight_layout()
        plt.show()


falki = {
    'Haar' : 'haar',
    'Daubechies 2' : 'db2',
    'Symlets 2' : 'sym2',
    'Coiflets 2' : 'coif2',
    'Biortogonalna' : 'bior1.3'
}

for label, wavelet in falki.items():
    f_slider = widgets.IntSlider(min=1, max=20, step=1, value=5, description="f [Hz]")
    display(widgets.interactive(decomposition_with_wavelet, label=widgets.fixed(label), wavelet=widgets.fixed(wavelet), f=f_slider))

interactive(children=(IntSlider(value=5, description='f [Hz]', max=20, min=1), Output()), _dom_classes=('widge…

interactive(children=(IntSlider(value=5, description='f [Hz]', max=20, min=1), Output()), _dom_classes=('widge…

interactive(children=(IntSlider(value=5, description='f [Hz]', max=20, min=1), Output()), _dom_classes=('widge…

interactive(children=(IntSlider(value=5, description='f [Hz]', max=20, min=1), Output()), _dom_classes=('widge…

interactive(children=(IntSlider(value=5, description='f [Hz]', max=20, min=1), Output()), _dom_classes=('widge…